In [3]:
pip install tensorflow

  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.4-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.71.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached h


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Directories for training and testing datasets
train_dir = '../data/spiral/training'
test_dir = '../data/spiral/testing'

# Image data generators for preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load the Xception model with pre-trained weights
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator
)


Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 68s 1us/step


c:\Users\HP 17\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 17s 4s/step - accuracy: 0.5618 - loss: 0.6951 - val_accuracy: 0.6667 - val_loss: 0.6326
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/step - accuracy: 0.7389 - loss: 0.6132 - val_accuracy: 0.7333 - val_loss: 0.5855
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.7561 - loss: 0.5697 - val_accuracy: 0.7333 - val_loss: 0.5388
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 5s/step - accuracy: 0.7896 - loss: 0.5113 - val_accuracy: 0.7333 - val_loss: 0.5059
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.6940 - loss: 0.5686 - val_accuracy: 0.7667 - val_loss: 0.4858
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 4s/step - accuracy: 0.8000 - loss: 0.4848 - val_accuracy: 0.7667 - val_loss: 0.4820
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - accuracy: 0.7218 - loss: 0.5019 - val_accuracy: 0.7667 - val_loss: 0.4738
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.7943 - loss: 0.4528 - val_accuracy: 0.7667 - val_loss: 0.4647
Epoch 9/20
3

In [11]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to the image you want to test
img_path = '../data/spiral/testing/healthy/V01HE01.png'  # <- replace this

# Load and preprocess the image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Rescale to match training preprocessing
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict
prediction = model.predict(img_array)

# Interpret prediction
if prediction[0][0] > 0.5:
    print("Predicted class: 1 (e.g. Parkinson's)")
else:
    print("Predicted class: 0 (e.g. Healthy)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Predicted class: 0 (e.g. Healthy)


In [12]:
model.save('my_model.h5')

In [3]:

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = False

for layer in base_model.layers[:-50]:  # Freeze all layers except the last 50
    layer.trainable = True
# Recompile the model for fine-tuning
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune the model
model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator
)

# Save the model
model.save('xception_spiral_classifier_v2.h5')

Epoch 1/50


KeyboardInterrupt: 